In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Load the dataset
df = pd.read_csv('NSL_training.csv')
# df = df[0:100]
# Encode categorical variables
le = LabelEncoder()
categorical_cols = ['protocol_type', 'service', 'flag', 'label']

for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

#Normalize
scaler = MinMaxScaler()
df.iloc[:, :-1] = scaler.fit_transform(df.iloc[:, :-1])

# Separate features and target
X = df.drop('label', axis=1)
y = df['label']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


C:\Users\maria\AppData\Local\Temp\ipykernel_10160\705672113.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:, :-1] = scaler.fit_transform(df.iloc[:, :-1])
C:\Users\maria\AppData\Local\Temp\ipykernel_10160\705672113.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5 1.  0.5 ... 0.5 0.5 0.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:, :-1] = scaler.fit_transform(df.iloc[:, :-1])
C:\Users\maria\AppData\Local\Temp\ipykernel_10160\705672113.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.28985507 0.63768116 0.71014493 ... 0.7826087  0.43478261 0.28985507]' has dtype incompatible wit

In [3]:
def objective_function(features_subset):
    if np.sum(features_subset) == 0:
        return 0  # Avoid empty feature set

    # Select features
    X_train_subset = X_train.iloc[:, features_subset]
    X_test_subset = X_test.iloc[:, features_subset]

    # Train a model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_subset, y_train)

    # Predict and calculate accuracy
    y_pred = model.predict(X_test_subset)
    accuracy = accuracy_score(y_test, y_pred)

    # We aim to maximize accuracy, so return negative for minimization
    return -accuracy

In [4]:
def firefly_algorithm(objective, n_fireflies, n_features, max_iter, alpha=0.5, beta0=1.0, gamma=1.0):
    # Initialize fireflies randomly
    fireflies = np.random.rand(n_fireflies, n_features)
    fireflies = (fireflies > 0.5).astype(int)

    # Evaluate fitness
    fitness = np.array([objective(firefly) for firefly in fireflies])

    # Keep track of the best firefly
    best_firefly = fireflies[np.argmin(fitness)].copy()
    best_fitness = np.min(fitness)

    for s in range(max_iter):
        print(s)
        for i in range(n_fireflies):
            for j in range(n_fireflies):
                if fitness[j] < fitness[i]:
                    # Calculate attractiveness
                    r = np.linalg.norm(fireflies[i] - fireflies[j])
                    beta = beta0 * np.exp(-gamma * r**2)

                    # Update firefly i towards firefly j
                    fireflies[i] = fireflies[i] + beta * (fireflies[j] - fireflies[i]) + alpha * (np.random.rand(n_features) - 0.5)
                    fireflies[i] = np.clip(fireflies[i], 0, 1)
                    fireflies[i] = (fireflies[i] >= 0.5).astype(int)

            # Evaluate new fitness
            new_fitness = objective(fireflies[i])
            if new_fitness < fitness[i]:
                fitness[i] = new_fitness
                if new_fitness < best_fitness:
                    best_fitness = new_fitness
                    best_firefly = fireflies[i].copy()

    return best_firefly, best_fitness

In [5]:
# Parameters
n_fireflies = 5
max_iter = 20
alpha = 0.5
beta0 = 1.0
gamma = 1.0

# Run the algorithm
best_firefly, best_fitness = firefly_algorithm(objective_function,
                                                n_fireflies,
                                                X_train.shape[1],
                                                max_iter,
                                                alpha=alpha,
                                                beta0=beta0,
                                                gamma=gamma)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [6]:
# Get the indices of selected features
selected_features = np.where(best_firefly == 1)[0]
print("Selected Feature Indices:", selected_features)

# Get feature names
feature_names = X.columns.tolist()
selected_feature_names = [feature_names[i] for i in selected_features]
print("Selected Feature Names:", selected_feature_names)

Selected Feature Indices: [ 1  2  7  8 15 16 19 20 21 23 25 27 29 30 32 33 35 36 37 38]
Selected Feature Names: ['protocol_type', 'service', 'wrong_fragment', 'urgent', 'num_root', 'num_file_creations', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'srv_count', 'srv_serror_rate', 'srv_rerror_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate']


In [7]:
# Select the best features
X_train_selected = X_train[selected_feature_names]
X_test_selected = X_test[selected_feature_names]

# Train a model with selected features
model_selected = RandomForestClassifier(n_estimators=100, random_state=42)
model_selected.fit(X_train_selected, y_train)

# Predict and calculate accuracy
y_pred_selected = model_selected.predict(X_test_selected)
accuracy_selected = accuracy_score(y_test, y_pred_selected)
print("Accuracy with Selected Features:", accuracy_selected)

Accuracy with Selected Features: 0.9913077991665012


In [8]:
selected_feature_names = selected_feature_names + ['label']
selected_features_df = df[selected_feature_names]
# selected_features_df
selected_features_df

,protocol_type,service,wrong_fragment,urgent,num_root,num_file_creations,num_outbound_cmds,is_host_login,is_guest_login,srv_count,...,srv_rerror_rate,diff_srv_rate,srv_diff_host_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,label
0,0.5,0.289855,0.0,0.0,0.0,0.0,0,0,0,0.003914,...,0.0,0.00,0.00,0.098039,0.17,0.17,0.00,0.00,0.00,11
1,1.0,0.637681,0.0,0.0,0.0,0.0,0,0,0,0.001957,...,0.0,0.15,0.00,0.003922,0.00,0.88,0.00,0.00,0.00,11
2,0.5,0.710145,0.0,0.0,0.0,0.0,0,0,0,0.011742,...,0.0,0.07,0.00,0.101961,0.10,0.00,0.00,1.00,1.00,9
3,0.5,0.347826,0.0,0.0,0.0,0.0,0,0,0,0.009785,...,0.0,0.00,0.00,1.000000,1.00,0.03,0.04,0.03,0.01,11
4,0.5,0.347826,0.0,0.0,0.0,0.0,0,0,0,0.062622,...,0.0,0.00,0.09,1.000000,1.00,0.00,0.00,0.00,0.00,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0.5,0.710145,0.0,0.0,0.0,0.0,0,0,0,0.048924,...,0.0,0.06,0.00,0.098039,0.10,0.00,0.00,1.00,1.00,9
125969,1.0,0.710145,0.0,0.0,0.0,0.0,0,0,0,0.003914,...,0.0,0.00,0.00,0.956863,0.96,0.01,0.00,0.00,0.00,11
125970,0.5,0.782609,0.0,0.0,0.0,0.0,0,0,0,0.001957,...,0.0,0.00,0.00,0.117647,0.12,0.00,0.00,0.72,0.00,11
125971,0.5,0.434783,0.0,0.0,0.0,0.0,0,0,0,0.015656,...,0.0,0.05,0.00,0.031373,0.03,0.00,0.00,1.00,1.00,9


In [9]:
output_file_path = 'selected_features_training.csv'
selected_features_df.to_csv(output_file_path, index=False)